<a href="https://colab.research.google.com/github/jeffvestal/elastic_jupyter_notebooks/blob/main/developer_genai_group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Install Libraries

In [12]:
!pip install -q streamlit openai elasticsearch
import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 22.5 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel --loglevel=error

## Elastic Cloud Credentials
Cloud ID -> Elastic Cloud Admin console

Elasticsearch API Keys - Generate in Kibana -> Stack Mgt


In [15]:
#test = getpass.getpass('test: ')
#os.environ['test'] = test
#print(os.getenv('test'))

os.environ['es_cloud_id'] = getpass.getpass('Enter Elastic Cloud ID:  ')
os.environ['es_api_id'] = getpass.getpass('Enter cluster API key ID:  ')
os.environ['es_api_key'] = getpass.getpass('Enter cluster API key:  ')

Enter Elastic Cloud ID:  ··········
Enter cluster API key ID:  ··········
Enter cluster API key:  ··········


## OpenAI / Azure OpenAI Credentials

In [16]:
#model = getpass.getpass('Enter model: ')
os.environ['model'] = "gpt-35-turbo"

## Getpass version of ENVs for colab
#openai.verify_ssl_certs = False

os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OPENAI_API_KEY: ')
os.environ['OPENAI_API_TYPE'] = getpass.getpass('Enter OPENAI_API_TYPE: ')
os.environ['OPENAI_API_BASE'] = getpass.getpass('Enter OPENAI_API_BASE: ')
os.environ['OPENAI_API_VERSION'] = getpass.getpass('Enter OPENAI_API_VERSION: ')
os.environ['OPENAI_API_ENGINE'] = getpass.getpass('Enter OPENAI_API_ENGINE: ')

Enter OPENAI_API_KEY: ··········
Enter OPENAI_API_TYPE: ··········
Enter OPENAI_API_BASE: ··········
Enter OPENAI_API_VERSION: ··········
Enter OPENAI_API_ENGINE: ··········


# Main Script
Running this cell will write a file named `app.py`

In [34]:
%%writefile app.py

import os
import streamlit as st
import openai
from elasticsearch import Elasticsearch
from string import Template


# Set Required Environment Variables

# Elastic Cloud
es_cloud_id = os.environ['es_cloud_id']
es_api_id = os.environ['es_api_id']
es_api_key = os.environ['es_api_key']


# OpenAI
openai.api_key = os.environ['OPENAI_API_KEY']
model = os.environ['model']

#Below is for Azure OpenAI
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_version = os.environ['OPENAI_API_VERSION']
engine = os.environ['OPENAI_API_ENGINE']
openai.verify_ssl_certs = False


# TODO show openai direct env



def es_connect(es_cloud_id, es_api_id, es_api_key):
    '''Connect to Elastic Cloud cluster'''
    es = Elasticsearch(cloud_id=es_cloud_id,
                   api_key=(es_api_id, es_api_key)
                   )
    print(es.info()) # should return cluster info
    return es

def search_bm25(query_text, es):
    '''Using ELSER -
       Search ElasticSearch index and return body and URL of the result'''

    # BM25 Query
    query = {
        "match": {
            "chunk": query_text
        }
    }

    fields= [
        "title",
        "url",
        "position",
        "body_content"
      ]

    collapse= {
    "field": "title.enum"
    }

    index = 'chunker'
    resp = es.search(index=index,
                     query=query,
                     fields=fields,
                     collapse=collapse,
                     size=1,
                     source=False)

    body = resp['hits']['hits'][0]['fields']['body_content'][0]
    url = resp['hits']['hits'][0]['fields']['url'][0]

    return body, url


def search_elser(query_text, es):
    '''Using ELSER -
       Search ElasticSearch index and return body and URL of the result'''

    # ELSER Query
    query = {
    "text_expansion": {
      "ml.inference.chunk_expanded.tokens": {
        "model_id": ".elser_model_1",
        "model_text": query_text
      }
    }
  }

    fields= [
        "title",
        "url",
        "position",
        "body_content"
      ]

    collapse= {
    "field": "title.enum"
    }

    index = 'chunker'
    resp = es.search(index=index,
                     query=query,
                     fields=fields,
                     collapse=collapse,
                     size=1,
                     source=False)

    body = resp['hits']['hits'][0]['fields']['body_content'][0]
    url = resp['hits']['hits'][0]['fields']['url'][0]

    return body, url

def search_knn(query_text, es):
    # Elasticsearch query (BM25) and kNN configuration for rrf hybrid search

    query = {
        "bool": {
            "must": [{
                "match": {
                    "title": {
                        "query": query_text
                    }
                }
            }],
            "filter": [{
              "term": {
                "url_path_dir3": "elasticsearch"
              }
            }]
        }
    }

    knn = [
    {
      "field": "chunk-vector",
      "k": 10,
      "num_candidates": 10,
      "filter": {
        "bool": {
          "filter": [
            {
              "range": {
                "chunklength": {
                  "gte": 0
                }
              }
            },
            {
              "term": {
                "url_path_dir3": "elasticsearch"
              }
            }
          ]
        }
      },
      "query_vector_builder": {
        "text_embedding": {
          "model_id": "sentence-transformers__msmarco-minilm-l-12-v3",
          "model_text": query_text
        }
      }
    }
  ]

    rank = {
       "rrf": {
       }
   }

#    collapse = {
#    "field": "title.enum"
#    }

    fields= [
        "title",
        "url",
        "position",
        "url_path_dir3",
        "body_content"
      ]

    index = 'chunker'
    resp = es.search(index=index,
                     query=query,
                     knn=knn,
                     rank=rank,
#                     collapse=collapse,
                     fields=fields,
                     size=10,
                     source=False)

    body = resp['hits']['hits'][0]['fields']['body_content'][0]
    url = resp['hits']['hits'][0]['fields']['url'][0]

    return body, url

def truncate_text(text, max_tokens):
    tokens = text.split()
    if len(tokens) <= max_tokens:
        return text

    return ' '.join(tokens[:max_tokens])

def chat_gpt(prompt, model="gpt-3.5-turbo", max_tokens=1024, max_context_tokens=4000, safety_margin=5, sys_content=None):
    '''# Generate a response from ChatGPT based on the given prompt'''

    # Truncate the prompt content to fit within the model's context length
    truncated_prompt = truncate_text(prompt, max_context_tokens - max_tokens - safety_margin)

    response = openai.ChatCompletion.create(engine=engine,
                                            temperature=0,
                                            messages=[{"role": "system", "content": sys_content},
                                                      {"role": "user", "content": truncated_prompt}]
                                           )

    return response["choices"][0]["message"]["content"]


def toLLM(resp,
          url,
          usr_prompt,
          sys_prompt,
          neg_resp):

    prompt_template = Template(usr_prompt)
    prompt_formatted = prompt_template.substitute(query=query, resp=resp, negResponse=negResponse)
    prompt_formatted
    answer = chat_gpt(prompt_formatted, sys_content=sys_prompt)

    # We don't need to return a reference URL if it wasn't useful
    if negResponse in answer:
        st.write(f"ChatGPT: {answer.strip()}")
    else:
        st.write(f"ChatGPT: {answer.strip()}\n\nDocs: {url}")


# MAIN
es = es_connect(es_cloud_id, es_api_id, es_api_key)

# Prompt Defaults
prompt_default = """Answer this question: $query
Using only the information from this Elastic Doc: $resp
If the answer is not contained in the supplied doc reply '$negResponse' and nothing else"""

system_default = 'You are a helpful assistant.'
neg_default = "I'm unable to answer the question based on the information I have from Elastic Docs."


# Main chat form
st.title("ElasticDocs GPT")

with st.form("chat_form"):
    query = st.text_input("You: ")

    # Inputs for system and User prompt override
    sys_prompt = st.text_area("create an alernative system prompt", placeholder=system_default, value=system_default)
    usr_prompt = st.text_area("create an alternative user prompt required -> \$query, \$resp, \$negResponse",
                             placeholder=prompt_default, value=prompt_default )

    # Default Response when criteria are not met
    negResponse = st.text_area("Create an alternative negative response", placeholder = neg_default, value=neg_default)

    # Query Submit Buttons
    col1, col2, col3 = st.columns(3)
    with col1:
        bm25_button = st.form_submit_button("Use BM25")
    with col2:
        elser_button = st.form_submit_button("Use ELSER")
    with col3:
        knn_button = st.form_submit_button("Use kNN")



if elser_button:
    resp, url = search_elser(query, es) # run ELSER query
    toLLM(resp, url, usr_prompt, sys_prompt, negResponse)

if knn_button:
    resp, url = search_knn(query, es) # run kNN hybrid query
    toLLM(resp, url, usr_prompt, sys_prompt, negResponse)

if bm25_button:
    resp, url = search_bm25(query, es) # run kNN hybrid query
    toLLM(resp, url, usr_prompt, sys_prompt, negResponse)




Overwriting app.py


# Streamlit
Running this cell will start local tunnel and generate a random URL

Copy the IP address on the first line then open the generated URL and paste it in the input box "Endpoint IP"

This will then start the Streamlit app

In [35]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.138.238.29
npx: installed 22 in 3.144s
your url is: https://every-shrimps-open.loca.lt
